<div style="text-align: right" align="right"><i>Roberto<br>December 1–, 2022</i></div>

# Advent of Code 2022

# Day 0: Preparations

Useful imports:

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import re

Each day's work will consist of three tasks, denoted by three bulleted section:
- **Input**: Parse the day's input file.  I will  use the function `parse(day, parser, sep)`, which:
   - Reads the input file for `day`.
   - Prints out the first few lines of the file (to remind me, and the notebook reader, what's in the file).
   - Breaks the file into a sequence of *entries* separated by `sep` (default newline).
   - Applies `parser` to each entry and returns the results as a tuple.
       - Useful parser functions include `ints`, `digits`, `atoms`, `words`, and the built-ins `int` and `str`.
- **Part 1**: Understand the day's instructions and:
   - Write code to compute the answer to Part 1.
   - Record the answer with the `answer` function, which also serves as a unit test when the notebook is re-run.
- **Part 2**: Understand the second part of the instructions and:
   - Write code and record `answer` for Part 2.
   
Occasionally I'll introduce a **Part 3** where I explore beyond the instructions.

Here are the helper functions for `answer` and `parse`:

In [2]:
def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was the expected answer."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

def parse(day, parser=str, sep='\n', print_lines=7) -> tuple:
    """Split the day's input file into entries separated by `sep`, and apply `parser` to each."""
    fname = f'data/{day}.txt'
    text  = open(fname).read()
    entries = mapt(parser, text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f'{fname} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:'
        dash = "-" * 100
        print(f'{dash}\n{head}\n{dash}')
        for line in lines:
            print(trunc(line))
        print(f'{dash}\nparse({day}) ➜ {len(entries)} entries:\n'
              f'{dash}\n{trunc(str(entries))}\n{dash}')
    return entries

def trunc(s: str, left=70, right=25, dots=' ... ') -> str: 
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
    dots = ' ... '
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]

In [3]:
Char = str # Intended as the type of a one-character string
Atom = Union[float, int, str]

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def words(text: str) -> List[str]:
    """A list of all the alphabetic words in text, ignoring non-letters."""
    return re.findall(r'[a-zA-Z]+', text)

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or symbol names) in text."""
    return mapt(atom, re.findall(r'[a-zA-Z_0-9.+-]+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text
    
def mapt(fn, *args) -> tuple:
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

A few additional  utility functions that I have used in the past:

In [4]:
def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)

def transpose(matrix) -> list: return list(zip(*matrix))

def nothing(*args) -> None: return None

cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

Some past puzzles involve (x, y) points on a rectangular grid, so I'll define  `Point` and `Grid`:

In [5]:
Point = Tuple[int, int] # (x, y) points on a grid

neighbors4 = ((0, 1), (1, 0), (0, -1), (-1, 0))               
neighbors8 = ((1, 1), (1, -1), (-1, 1), (-1, -1)) + neighbors4

class Grid(dict):
    """A 2D grid, implemented as a mapping of {(x, y): cell_contents}."""
    def __init__(self, mapping=(), rows=(), neighbors=neighbors4):
        """Initialize with, e.g., either `mapping={(0, 0): 1, (1, 0): 2, ...}`,
        or `rows=[(1, 2, 3), (4, 5, 6)].
        `neighbors` is a collection of (dx, dy) deltas to neighboring points.`"""
        self.update(mapping if mapping else
                    {(x, y): val 
                     for y, row in enumerate(rows) 
                     for x, val in enumerate(row)})
        self.width  = max(x for x, y in self) + 1
        self.height = max(y for x, y in self) + 1
        self.deltas = neighbors
        
    def copy(self) -> Grid: return Grid(self, neighbors=self.deltas)
    
    def neighbors(self, point) -> List[Point]:
        """Points on the grid that neighbor `point`."""
        x, y = point
        return [(x+dx, y+dy) for (dx, dy) in self.deltas 
                if (x+dx, y+dy) in self]
    
    def to_rows(self) -> List[List[object]]:
        """The contents of the grid in a rectangular list of lists."""
        return [[self[x, y] for x in range(self.width)]
                for y in range(self.height)]

# Day 2:

In [6]:
in2 = parse(2,lambda x: x.split())


----------------------------------------------------------------------------------------------------
data/2.txt ➜ 9999 chars, 2500 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
B Y
A X
B Y
A Y
A Z
B Y
B Z
----------------------------------------------------------------------------------------------------
parse(2) ➜ 2500 entries:
----------------------------------------------------------------------------------------------------
(['B', 'Y'], ['A', 'X'], ['B', 'Y'], ['A', 'Y'], ['A', 'Z'], ['B', 'Y' ... , ['A', 'X'], ['B', 'Z'])
----------------------------------------------------------------------------------------------------


In [7]:

def getPoints(a,b):
    return getPointsByShape(b) + getPointsByPlay(a,b)


def getPointsByShape(s):
    if (s == 'X'):
        return 1
    if(s == 'Y'):
        return 2
    if(s == 'Z'):
        return 3

def getPointsByPlay(a,b):
    if(a == 'A' and b == 'X' or a == 'B' and b == 'Y' or a == 'C' and b == 'Z'):
        return 3
    if(a == 'A' and b == 'Y'):
        return 6
    if(a == 'B' and b == 'Z'):
        return 6
    if(a == 'C' and b == 'X'):
        return 6
    return 0
    

sum(getPoints(p[0], p[1]) for p in in2)


12740

In [8]:

def getPoints(a,b):
    return getPointsByPlay(a,b)

def getPointsByPlay(a,b):
    if(b == 'X'):
        s = getShapeToLoose(a)
        return getPointsByShape(s)
    if(b == 'Y'):
        s = getShapeToDraw(a)
        return getPointsByShape(s) + 3
    if(b == 'Z'):
        s = getShapeToWin(a)
        return getPointsByShape(s) + 6
    raise Exception(f'Invalid value for b {b}')

def getPointsByShape(s):
    if (s == 'A'):
        return 1
    if(s == 'B'):
        return 2
    if(s == 'C'):
        return 3
    # next line should throw an error
    raise Exception('Invalid shape')
    
def getShapeToWin(a):
    if(a == 'A'):
        return 'B'
    if(a == 'B'):
        return 'C'
    return 'A'
def getShapeToLoose(a):
    if(a == 'A'):
        return 'C'
    if(a == 'B'):
        return 'A'
    return 'B'
def getShapeToDraw(a):
    return a

sum(getPoints(p[0], p[1]) for p in in2)

11980

# Day 3:

In [9]:
in3 = parse(3,lambda x: [x[0:int(len(x)/2)],x[int(len(x)/2):]])

----------------------------------------------------------------------------------------------------
data/3.txt ➜ 9589 chars, 300 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
wgqJtbJMqZVTwWPZZT
LHcTGHQhzrTzBsZFPHFZWFFs
RnLRClzGzRGLGLGCNRjTMjJfgmffSffMqNgp
WPLgsfLmLgqZvZgSRR
RbwHdbDdQFFFMvvMjbhqhZZS
lzTdldBDszfGcRsr
ZjnhJjMjnbdnbHdFLmmfFLmnCCWFFl
----------------------------------------------------------------------------------------------------
parse(3) ➜ 300 entries:
----------------------------------------------------------------------------------------------------
(['wgqJtbJMq', 'ZVTwWPZZT'], ['LHcTGHQhzrTz', 'BsZFPHFZWFFs'], ['RnLRC ... 'zrDzSSzfgTPqTSTTtSPgt'])
----------------------------------------------------------------------------------------------------


In [10]:
def getValue(c):
    n = ord(c)
    if(n >= 97):
        return n - 96
    return n - 38

def getShared(c1,c2):
    shared = c1 & c2
    return shared.most_common()[0][0]

In [11]:
c = [[Counter(x),Counter(y)] for x,y in in3]
result = [getValue(getShared(x[0],x[1])) for x in c]
sum(result)

7727

In [12]:
in3Test = parse("test_3")
in3 = parse(3)

----------------------------------------------------------------------------------------------------
data/test_3.txt ➜ 149 chars, 6 lines; first 6 lines:
----------------------------------------------------------------------------------------------------
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
----------------------------------------------------------------------------------------------------
parse(test_3) ➜ 6 entries:
----------------------------------------------------------------------------------------------------
('vJrwpWtwJgWrhcsFMMfFFhFp', 'jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL', 'Pmmd ... rZsJsPPZsGzwwsLwLmpwMDw')
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
data/3.txt ➜ 9589 chars, 300 lines; first 7 lines:
------------

In [13]:
acc = []
t = []
for x,_ in enumerate(in3):
    t.append(Counter(in3[x]))
    if(x%3 == 2):
        acc.append(t)
        t = []
# print(acc)

def getSharedThree(c1,c2,c3):
    shared = c1 & c2 & c3
    return shared.most_common()[0][0]

result = [getValue(getSharedThree(x[0],x[1],x[2])) for x in acc]
sum(result)


2609

# Day 4:

In [76]:
def contained(a,b):
    return a[0] <= b[0] and a[1] >= b[1]

def someOverlap(a,b):
    return (a[0] <= b[0] and a[1] >= b[0]) 

def getInput(x):
    entries = x.split(",")
    entry1 = entries[0].split("-")
    entry2 = entries[1].split("-")
    return ((int(entry1[0]),int(entry1[1])),(int(entry2[0]),int(entry2[1])))


in4 = parse("4", getInput )

----------------------------------------------------------------------------------------------------
data/4.txt ➜ 11369 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
57-93,9-57
55-55,55-83
55-88,78-88
24-24,24-95
7-92,8-93
25-84,84-85
62-85,62-85
----------------------------------------------------------------------------------------------------
parse(4) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
(((57, 93), (9, 57)), ((55, 55), (55, 83)), ((55, 88), (78, 88)), ((24 ... )), ((22, 23), (22, 60)))
----------------------------------------------------------------------------------------------------


In [77]:
quantify(contained(a,b) or contained(b,a) for a,b in in4)

471

In [78]:
quantify(someOverlap(a,b) or someOverlap(b,a) for a,b in in4)

888